In [40]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [43]:
# Assumptions of input data - (shape of input tensor =  (n * 32 * 32 * 1))
#n = number of training samples
image_size =  32
num_labels = 10
num_channels = 1 # assuming grayscale

# helper methods
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def output_size_no_pool(input_size, filter_size, padding, conv_stride):
    if padding == 'same':
        padding = -1.00
    elif padding == 'valid':
        padding = 0.00
    else:
        return None
    output_1 = float(((input_size - filter_size- 2*padding)/conv_stride) + 1.00)
    return int(np.ceil(output_1))

patch_size = 3
final_image_size = output_size_no_pool(image_size, patch_size, padding='valid', conv_stride = 2)
print(final_image_size)


16


In [52]:
batch_size = 1 # number of samples to train at a given iteration

depth = 4 # 16*16*4
num_hidden = 10

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    #conv_filter = (3* 3 * 4) which gives (16 * 16 * 4)
    # Convolution 1 Layer
    # Input channels: num_channels = 1
    # Output channels: depth = 4
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    # Fully Connected Layer
    layer2_weights = tf.Variable(tf.truncated_normal([final_image_size * final_image_size * depth, num_hidden], stddev = 0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        # First Convolutional Layer
        conv_1 = tf.nn.conv2d(data, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
        #relu
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        
        # Full Connected Layer
        shape = hidden_1.get_shape().as_list()
        reshape = tf.reshape(hidden_1, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer2_weights) + layer2_biases)
        
        return tf.matmul(hidden, layer3_weights) + layer3_biases
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    '''Optimizer'''
    # Learning rate of 0.05
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
